## Support Vector Classifier for emotion, based on 4 simple features

Binary classification for each emotion. Trained with k fold cross validation and a loop over multiple shufflings of the data. Accuracy around 60%. Random is 50%.

Balences data to account for imbalanced data (some emotions are not reported as often as others)


##### Observe: data poorly seperable with SVC linear classifier

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import copy

In [4]:
# Data import

ZC = np.load('../preprocessing/SimpleFeatures/zeroCrossings_frame200ms_hop100ms.npy')
SC = np.load('../preprocessing/SimpleFeatures/spectralCentroid_frame200ms_hop100ms.npy')
SV = np.load('../preprocessing/SimpleFeatures/spectralVariance_frame200ms_hop100ms.npy')
ST = np.load('../preprocessing/SimpleFeatures/staticTempoLibrosa.npy')

labels = pd.read_csv('../preprocessing/labels.csv')

In [5]:
# Only keep the emotion labels 
# These correspond to the average number each specific emotion was reported

# For classifying we threshold these averages at 0.5. 
# Average > 0.5: this emotion corresponds to this song
# Average < 0.5: emotion does not belong to this song

labels = labels.drop(['genre',
             'var_amazement',
             'var_solemnity',
             'var_tenderness',
             'var_nostalgia',
             'var_calmness',
             'var_power',
             'var_joyful_activation',
             'var_tension',
             'var_sadness',
             'mood',
             'var_mood',
             'liked',
             'var_liked',
             'disliked',
             'var_disliked',
             'age',
             'var_age',
             'gender',
             'var_gender',
             'number_of_observations',
             'track id'             
            ], axis=1)

emotions = list(labels)

In [6]:
np.mean(labels)

amazement            0.134012
solemnity            0.195961
tenderness           0.176911
nostalgia            0.254279
calmness             0.299257
power                0.190244
joyful_activation    0.261896
tension              0.237685
sadness              0.185288
dtype: float64

------------------------------

In [7]:
# Take the median over all frames for each song to have comparable dimensions

ZC = np.nanmedian(ZC, axis=1)
SC = np.nanmedian(SC, axis=1)
SV = np.nanmedian(SV, axis=1)

In [8]:
# OR take the mean in a certain interval

# ZC = np.mean(ZC[:, 100:200], axis=1)
# SC = np.mean(SC[:, 100:200], axis=1)
# SV = np.mean(SV[:, 2:4], axis=1)

In [9]:
# Make dataframe

features_dict_medians = {'ZC': ZC,
                         'SC': SC,
                         'SV': SV,
                         'StaticTempo': ST}
features = pd.DataFrame(data=features_dict_medians)

In [10]:
# Shuffle

def shuffleData(features, labels):
    n_data = len(features)
    idx = np.random.permutation(n_data)
    features, labels = features.reindex(idx), labels.reindex(idx)
    return features, labels

In [11]:
# K fold Crossvalidation

def kFoldConfusionMatrix(k, features, labels):

    kfold_train_metrics = []
    kfold_test_metrics = []

    confusion_Matrix_total = np.zeros((2, 2), dtype= 'int')

    # # Drop NaN values
    # NaNidx = np.where(features.isnull().any(axis=1).tolist())[0]
    # # NaNidx = np.isnan(features)[np.isnan(features) == 1].stack().index.tolist()

    # total_input = features.drop(NaNidx)
    # total_labels = labels.drop(NaNidx)

    total_input = features
    total_labels = labels

    cv = KFold(n_splits=k)

    for train_index, test_index in cv.split(total_input):        
        train_df, train_labels = total_input.iloc[train_index], total_labels.iloc[train_index]
        test_df, test_labels = total_input.iloc[test_index], total_labels.iloc[test_index]

        # Standardizing data
        mean = train_df.mean()
        std = train_df.std()

        train_df = (train_df - mean) / std
        test_df = (test_df - mean) / std

        # Classifier
        clf = svm.SVC(kernel='linear', class_weight='balanced')
        clf.fit(train_df, train_labels)

        kfold_train_metrics.append(clf.score(train_df, train_labels))
        kfold_test_metrics.append(clf.score(test_df, test_labels))

        assert np.shape(clf.predict(test_df)) == np.shape(test_labels)
        
        CM = confusion_matrix(test_labels, clf.predict(test_df), labels= [0,1])
        
        assert np.shape(CM) == (2,2)

        confusion_Matrix_total += CM
    return confusion_Matrix_total

In [12]:
# Doing the actual training of the support vector classifier over multple rounds of shuffeling
threshold = 0.5

N_emotions = 9
N_features = 4

confusion_Matrix = np.zeros((N_emotions, 2, 2))

N_shuffles = 5
k = 10
accuracies = np.zeros((N_emotions, N_shuffles))

for e_idx, emotion in enumerate(emotions):
    for Siter in range(N_shuffles):
        thresholdedLabel = copy.deepcopy(labels[emotion])
        thresholdedLabel[thresholdedLabel >= threshold] = int(1)
        thresholdedLabel[thresholdedLabel <  threshold] = int(0)

        features, thresholdedLabel = shuffleData(features, thresholdedLabel)

        confusion_Matrix[e_idx, :, :] += kFoldConfusionMatrix(k, features, thresholdedLabel)

        accuracies[e_idx, Siter] = np.sum(np.eye(2)*confusion_Matrix[e_idx, :, :])/np.sum(confusion_Matrix[e_idx, :, :])

In [13]:
# Confusion matrix
for e_idx, emotion in enumerate(emotions):
    print(f'EMOTION {emotion}: \n')
    print(f'Accuracy: {np.mean(accuracies[e_idx, :])} +- {np.std(accuracies[e_idx, :])} \n')

    CMATRIX = confusion_Matrix[e_idx, :, :]
    print(CMATRIX)
    print('\n\n')

EMOTION amazement: 

Accuracy: 0.807625 +- 0.0028613807855649197 

[[1.619e+03 3.660e+02]
 [1.400e+01 1.000e+00]]



EMOTION solemnity: 

Accuracy: 0.5016583333333333 +- 0.0028687008286756755 

[[944. 961.]
 [ 42.  53.]]



EMOTION tenderness: 

Accuracy: 0.49672499999999997 +- 0.007019793444254625 

[[925. 940.]
 [ 54.  81.]]



EMOTION nostalgia: 

Accuracy: 0.4138916666666666 +- 0.0043056487445112205 

[[ 685. 1085.]
 [  89.  141.]]



EMOTION calmness: 

Accuracy: 0.42334999999999995 +- 0.007151223671512462 

[[586. 969.]
 [170. 275.]]



EMOTION power: 

Accuracy: 0.5889916666666666 +- 0.012588199941920933 

[[1079.  751.]
 [  92.   78.]]



EMOTION joyful_activation: 

Accuracy: 0.5789166666666666 +- 0.0009718253158075249 

[[950. 635.]
 [210. 205.]]



EMOTION tension: 

Accuracy: 0.5599833333333334 +- 0.0014723375216905204 

[[978. 797.]
 [ 84. 141.]]



EMOTION sadness: 

Accuracy: 0.46797500000000003 +- 0.009146174063508728 

[[ 868. 1027.]
 [  37.   68.]]





In [ ]:
# # Confusion matrix
# for e_idx, emotion in enumerate(emotions):
#     print(f'EMOTION {emotion}: \n')
#     print(f'Accuracy: {np.mean(accuracies[e_idx, :])} +- {np.std(accuracies[e_idx, :])} \n')

#     fig,ax = plt.subplots()
#     CMATRIX = confusion_Matrix[e_idx, :, :]
#     print(CMATRIX)
#     print('\n\n')
#     CM = ConfusionMatrixDisplay(CMATRIX)
#     CM.plot(ax=ax)
#     ax.set_title(f'{emotion}')